In [9]:
import tkinter as tk
from tkinter import filedialog
import os
import subprocess

class CheckBoxWithField(tk.Frame):
    def __init__(self,master=None,prompt="",option="",entry_label=""):
        super().__init__(master)
        self.option = option
        self.prompt = prompt
        self.is_checked=tk.IntVar(self,0)
        self.pack(fill="both")
        self.CHECKBOX = tk.Checkbutton(self, text=self.prompt, variable=self.is_checked,command=self.button_checked)
        self.CHECKBOX.pack(side="left")
        self.spacing = tk.Label(self,text="\t")
        self.spacing.pack(side="left")
       
        self.LABEL = tk.Label(self,text=entry_label)
        self.TEXT = tk.Entry(self,state="disabled")
        
    def button_checked(self):
        if self.is_checked.get():
            self.TEXT.configure(state="normal")
            self.TEXT.pack(side="right")
            self.LABEL.pack(side="right")
        else:
            self.TEXT.configure(state="disabled")
            self.LABEL.pack_forget()
            self.TEXT.pack_forget()
        
    def command_string(self):
        if self.is_checked.get():
            return f"{self.option} {self.TEXT.get()} "
        return ""


class RadioButtonBoolWithTextEntryFrame(tk.Frame):
    def __init__(self,name,cond_text,master=None):
        super().__init__(master)
        self.name     = name
        self.cond_text= cond_text
        self.entry = tk.Entry(self)
        self.custom_entry = ""
        self.last_val = ""
        self.custom_text = tk.Label(self,text=self.cond_text)
        self.values   = {"Yes":True,"No":False}
        self.boolean  = tk.BooleanVar(self,False)
        self.pack(fill='both')
        tk.Label(self, text=self.name).pack(side="left")
        for (text, value) in self.values.items():
            tk.Radiobutton(self, text = text, variable = self.boolean,
                value = value,command=self.update).pack(fill="x",side="right", ipady = 2,ipadx=2)
    def update(self):
        if self.boolean.get():
            self.entry.pack_forget()
            self.custom_text.pack_forget()
            self.entry.pack(side="right")
            self.custom_text.pack(side="right")
            self.entry.delete(10,tk.END)
            self.entry.insert(10,self.last_val)
        else:
            self.entry.pack_forget()
            self.custom_text.pack_forget()
            self.custom_entry = ""
    def get(self):
        self.custom_entry = self.entry.get()
        self.last_val     = self.entry.get()
        return self.boolean.get()


def ADD_TITLE(frame,title):
    frame.TITLE = tk.Label(frame,text=title)
    frame.TITLE.pack(side="top")

def ADD_SELF_DESTRUCT(frame):
    frame.SELF_DESTRUCT = tk.Button(frame,text="REMOVE ANALYSIS",fg="red",command=frame.destroy)
    frame.SELF_DESTRUCT.pack(side="right")

def ADD_DATASET_NAME(frame,default_text=""):
    frame.DATASET_FRAME = tk.Frame(frame)
    frame.DATASET_FRAME.pack(side="top",fill="both")
    frame.dataset_label = tk.Label(frame.DATASET_FRAME,text="Dataset Name")
    frame.dataset_label.pack(side="left")
    frame.dataset_entry = tk.Entry(frame.DATASET_FRAME)
    frame.dataset_entry.insert(0, default_text)
    frame.dataset_entry.pack(side="right")

def FILE_OUTPUT_FRAME(frame,default_text=""):
    frame.FILE_OUTPUT=tk.Frame(frame)
    frame.FILE_OUTPUT.pack(side="top",fill='both')
    frame.file_label = tk.Label(frame.FILE_OUTPUT,text="Output filename")
    frame.file_label.pack(side="left")
    frame.file_entry = tk.Entry(frame.FILE_OUTPUT)
    frame.file_entry.insert(0, default_text)
    frame.file_entry.pack(side="right")

def SELECTION_MASK(frame,label,default_text=""):
    FRAME=tk.Frame(frame)
    FRAME.pack(side="top",fill='both')
    rmsd_label = tk.Label(FRAME,text=label)
    rmsd_label.pack(side="left")
    selection_mask = tk.Entry(FRAME)
    selection_mask.insert(0, default_text)
    selection_mask.pack(side="right")
    return FRAME


### MAIN CLUSTERING FRAME
class Clustering(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Clustering")
        self.options=["SELECT CLUSTERING ALGORITHM","Heirarchical Agglomerative", "DBSCAN", "Density Peaks","K-means"]
        self.algo_type = tk.StringVar()
        self.algo_type.set("SELECT ANALYSIS")
        
        ### Select Algorithm Frame
        self.ALGORITHM = tk.Frame(self)
        self.ALGORITHM.pack()
        self.algo_choice = tk.OptionMenu(self.ALGORITHM ,self.algo_type, *self.options,command=self.update_working)
        self.algo_choice.pack(side="top")
        ### Chosen Algorithm Frame
        self.WORKING = tk.Frame(self.ALGORITHM)
        self.WORKING.pack()
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def update_working(self):
        self.WORKING.pack_forget()
        
        
        self.WORKING.pack(side="top")
        
    def command_string(self):
        fileout = self.file_entry.get()
        



In [10]:
### HeirAgglo
# [hieragglo [epsilon <e>] [clusters <n>] [linkage|averagelinkage|complete]
#       [epsilonplot <file>] [includesieved_cdist]]
class HeirAgglo(tk.Frame):
    def __init__(self,master):
        super().__init__(master)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"HeirAgglo Clustering")
        
        ### Options Frames
        # Epsilon
        self.EPSILON     = SELECTION_MASK(self,"Epsilon",default_text="0.5")
        # Clusters
        self.CLUSTERS    = SELECTION_MASK(self,"Clusters",default_text="10")
        # linkage options
        self.LINKAGE_OPT = tk.Frame(self)
        self.LINKAGE_OPT.pack(side="top")
        self.linkage_opt = tk.StringVar(self,"linkage")
        self.LINKAGE_OPT.pack(fill='both')
        tk.Label(self.LINKAGE_OPT, text="").pack(side="left")
        for (text, value) in {"Linkage":"linkage","Average Linkage":"averagelinkage","Complete":"complete"}.items():
            tk.Radiobutton(self.LINKAGE_OPT, text = text, variable = self.linkage_opt,
                value = value).pack(fill="x",side="right", ipady = 2,ipadx=2)

        # Save Epsilon Plot        
        self.EPS_PLOT    = RadioButtonBoolWithTextEntryFrame("Save Epsilon Plot?","Filename",self)

        # Include Sieved C-dist
        self.INCL_SIEVED = tk.Frame(self)
        self.INCL_SIEVED.pack(side="top")
        self.includesieved = tk.BooleanVar(self,False)
        tk.Label(self.INCL_SIEVED, text="Include Sieved C-distance?").pack(side="left")
        for (text, value) in {"Yes":True,"No":False}.items():
            tk.Radiobutton(self.INCL_SIEVED, text = text, variable = self.includesieved,
                value = value).pack(fill="x",side="right", ipady = 2,ipadx=2)

        
    def command_string(self):
        full_string  =  "heiragglo "
        full_string += f"epsilon {self.EPSILON.selection_mask.get()} "
        full_string += f"clusters {self.CLUSTERS.selection_mask.get()} "
        full_string += f"{self.linkage_opt.get()} "
        if self.EPS_PLOT.get():
            full_string += f"epsilonplot {self.EPS_PLOT.custom_entry} "
        if self.includesieved.get():
            full_string += "includesieved_cdist "
        return full_string

In [11]:
### DBScan
# [dbscan minpoints <n> epsilon <e> 
# [sievetoframe] [kdist <k> [kfile <prefix>]]]

class DBScan(tk.Frame):
    def __init__(self,master):
        super().__init__(master)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"DBSCAN Clustering")
        
        ### Variables
        self.minpoints = tk.IntVar(self,10)
        self.epsilon   = tk.DoubleVar(self,0.5)
        self.sievetoframe = tk.BooleanVar(self,False)
        self.kdist = tk.BooleanVar(self,False)
        self.kdistfile = tk.StringVar(self,"KDistPlot.dat")
        
        ### Options Frame
        

In [12]:
### DPeaks
# [dpeaks epsilon <e> [noise] [dvdfile <density_vs_dist_file>]
# 	  [choosepoints {manual | auto}]
# 	  [distancecut <distcut>] [densitycut <densitycut>]
# 	  [runavg <runavg_file>] [deltafile <file>] [gauss]]


class DPeaks(tk.Frame):
    def __init__(self,master):
        super().__init__(master)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Density Peaks Clustering")
        
        ### Variables
        self.epsilon   = tk.DoubleVar(self,0.5)
        self.noise = tk.BooleanVar(self,False)
        self.dvd = tk.BooleanVar(self,False)
        self.dvdfile = tk.StringVar(self,"Density_vs_Distance.dat")
        self.choosepoints = tk.Stringvar(self,"auto")
        self.distancecut = tk.DoubleVar(self,5.0)
        self.densitycut = tk.DoubleVar(self,10.0)
        self.runavg =tk.BooleanVar(self,False)
        self.runavgfile = tk.StringVar(self,"Running_Average.dat")
        self.delta = tk.BooleanVar(self,False)
        self.deltafile = tk.StringVar(self,"Delta.dat")
        self.gauss = tk.BooleanVar(self,False)
        ### Options Frame

In [13]:
### K-means
# [kmeans clusters <n> [randompoint [kseed <seed>]] [maxit <iterations>]
#     [{readtxt|readinfo} infofile <file>]

class Kmeans(tk.Frame):
    def __init__(self,master):
        super().__init__(master)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"K-means Clustering")
        
        ### Variables
        self.clusters   = tk.IntVar(self,10)
        self.randompoint = tk.BooleanVar(self,False)
        self.kseed = tk.BooleanVar(self,False)
        self.seed = tk.DoubleVar(self,1.0)
        self.maxit = tk.BooleanVar(self,False)
        self.iterations = tk.IntVar(self,1000)
        self.readtxt = tk.StringVar(self,"readinfo")
        self.infofile = tk.BooleanVar(self,False)
        self.infofilename = tk.StringVar(self,"infofile.dat")
        
        ### Options Frame

In [24]:

root = tk.Tk()
root.title("CPPTRAJ Analysis Generator")
app = CheckBoxWithField(root,prompt = "prompt?",option="prompt",entry_label="entry")
app.mainloop()